# Playground

## GNN
This code receives multiple timeseries, transforms them into graphs, and then applies a GNN to them.
The graph embeddings are then used for downstream tasks.

In [2]:
%reload_ext autoreload
%autoreload 3

import pytorch_lightning as pl
from src.b2bnet import B2BNetModel, RandomDataModule, OtkaDataModule
from pytorch_lightning.loggers import TensorBoardLogger

In [6]:
# VISUALIZE model's graph
datamodule = OtkaDataModule()
model = B2BNetModel(input_size=59, n_timesteps=100, n_cls_labels=2, text_dim=1)

# TRAINING
trainer = pl.Trainer(max_epochs=2, accelerator='cpu', log_every_n_steps=1)

trainer.fit(model, datamodule=datamodule)

GPU available: True (mps), used: False
TPU available: False, using: 0 TPU cores
IPU available: False, using: 0 IPUs
HPU available: False, using: 0 HPUs
/opt/homebrew/Caskroom/miniforge/base/envs/b2bnet/lib/python3.10/site-packages/pytorch_lightning/trainer/setup.py:201: UserWarning: MPS available but not used. Set `accelerator` and `devices` using `Trainer(accelerator='mps', devices=1)`.
  rank_zero_warn(

  | Name        | Type             | Params
-------------------------------------------------
0 | encoder     | RNN              | 1.3 K 
1 | fc          | Linear           | 6     
2 | decoder     | RNN              | 704 K 
3 | b2b         | RNN              | 704 K 
4 | fc_text     | Linear           | 3     
5 | loss_cls    | CrossEntropyLoss | 0     
6 | loss_reconn | MSELoss          | 0     
7 | loss_b2b    | MSELoss          | 0     
8 | loss_text   | MSELoss          | 0     
-------------------------------------------------
1.4 M     Trainable params
0         Non-trainable

Sanity Checking: 0it [00:00, ?it/s]

/opt/homebrew/Caskroom/miniforge/base/envs/b2bnet/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:430: PossibleUserWarning: The dataloader, val_dataloader, does not have many workers which may be a bottleneck. Consider increasing the value of the `num_workers` argument` (try 8 which is the number of cpus on this machine) in the `DataLoader` init to improve performance.
  rank_zero_warn(
/opt/homebrew/Caskroom/miniforge/base/envs/b2bnet/lib/python3.10/site-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([11])) that is different to the input size (torch.Size([11, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/opt/homebrew/Caskroom/miniforge/base/envs/b2bnet/lib/python3.10/site-packages/pytorch_lightning/trainer/connectors/data_connector.py:430: PossibleUserWarning: The dataloader, train_dataload

Training: 0it [00:00, ?it/s]

/opt/homebrew/Caskroom/miniforge/base/envs/b2bnet/lib/python3.10/site-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([32])) that is different to the input size (torch.Size([32, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)
/opt/homebrew/Caskroom/miniforge/base/envs/b2bnet/lib/python3.10/site-packages/torch/nn/modules/loss.py:536: UserWarning: Using a target size (torch.Size([8])) that is different to the input size (torch.Size([8, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

`Trainer.fit` stopped: `max_epochs=2` reached.


## TCN

In [10]:
%reload_ext autoreload
%autoreload 3

import torch
from src.b2bnet import TCN
from torchinfo import summary

In [8]:
batch_size = 32
n_timesteps = 40000
n_features = 59
X = torch.randn(batch_size, n_timesteps, n_features).permute(0, 2, 1)
model = TCN(n_timesteps, output_length=128, n_features=n_features, kernel_size=2, dilation_base=2)
model(X).shape

torch.Size([32, 59, 40000])

In [11]:
summary(model, input_size=(batch_size, n_features, n_timesteps))

/Users/morteza/micromamba/envs/b2bnet/lib/python3.10/site-packages/torchinfo/torchinfo.py:477: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  action_fn=lambda data: sys.getsizeof(data.storage()),
/Users/morteza/micromamba/envs/b2bnet/lib/python3.10/site-packages/torch/storage.py:665: UserWarning: TypedStorage is deprecated. It will be removed in the future and UntypedStorage will be the only storage class. This should only matter to you if you are using storages directly.  To access UntypedStorage directly, use tensor.untyped_storage() instead of tensor.storage()
  return super().__sizeof__() + self.nbytes()


Layer (type:depth-idx)                   Output Shape              Param #
TCN                                      [32, 59, 40000]           --
├─Sequential: 1-1                        [32, 59, 40000]           --
│    └─TCNBlock: 2-1                     [32, 59, 40000]           --
│    │    └─Sequential: 3-1              [32, 59, 40000]           7,021
│    └─TCNBlock: 2-2                     [32, 59, 40000]           --
│    │    └─Sequential: 3-2              [32, 59, 40000]           7,021
│    └─TCNBlock: 2-3                     [32, 59, 40000]           --
│    │    └─Sequential: 3-3              [32, 59, 40000]           7,021
│    └─TCNBlock: 2-4                     [32, 59, 40000]           --
│    │    └─Sequential: 3-4              [32, 59, 40000]           7,021
│    └─TCNBlock: 2-5                     [32, 59, 40000]           --
│    │    └─Sequential: 3-5              [32, 59, 40000]           7,021
│    └─TCNBlock: 2-6                     [32, 59, 40000]           --
